### NOTES

We represent the solution in the form of genome which is a list of 0s and 1s.<br/>
0 indicating that the ingredient is excluded in the pizza recipe <br/>
1 indicating that the ingredient is included in the pizza recipe<br/>

The genome can be remapped to list of ingredients using the function `genome_to_ingredients`

Example:<br/>
Assuming `allIngredients = [tomato, onion, carrot, cabbage]`
<br/>
The genome `[0, 1, 1, 0]` represents `[onion, carrot]`
<br/>
`genome_to_ingredients([0, 1, 1, 0])` returns `[onion, carrot]`


NOTE: The length of the genome will be same as that of the total number of ingredients


In [12]:
# Python Version: 3.9 or above

# Requirements
# !pip install pygad
# !pip install numpy

Note: you may need to restart the kernel to use updated packages.


The filename, directory name, or volume label syntax is incorrect.


Note: you may need to restart the kernel to use updated packages.


The filename, directory name, or volume label syntax is incorrect.


In [1]:
from random import choice, choices, randint, randrange, random
from math import log10
import os
import time
import pygad
import json
import numpy as np

In [2]:
# Parses the input file and returns a list of (likes, dislikes) of each client
# Example: For input 'a_an_example.in.txt', the function returns
# [[[cheese, pepper], []], [[basil], [pineapple]], [[mushrooms, tomatoes], [basil]]]

def parse_input(filename: str):
    with open(filename, "r") as f:
        data = f.read()
    data = [i for i in data.split("\n") if i.strip() != ""]

    data = data[1:]
    client_data = [[data[i], data[i + 1]] for i in range(0, len(data), 2)]

    for i in range(len(client_data)):
        likes, dislikes = client_data[i]
        likes = set(likes.split(" ")[1:])
        dislikes = set(dislikes.split(" ")[1:] or [])
        client_data[i] = [likes, dislikes]
    return client_data


In [3]:
def score_pizza(pizza_ingredients: set[str]):
    "Used for scoring the solution found"
    pizza_ingredients = set(pizza_ingredients)
    score = 0
    global client_data
    for client_info in client_data:
        client_likes, client_dislikes = client_info
        client_likes_pizza = True

        # all the ingredients they like are on the pizza
        for liked_ingredient in client_likes:
            if liked_ingredient not in pizza_ingredients:
                client_likes_pizza = False
                break

        # none of the ingredients they dislike are on the pizza
        if client_likes_pizza:
            for disliked_ingredient in client_dislikes:
                if disliked_ingredient in pizza_ingredients:
                    client_likes_pizza = False
                    break
        if client_likes_pizza:
            score += 1
    return score


In [5]:
def getAllIngredients():
    "Get all ingredients used in input dataset"
    global client_data
    allIngredients = set()
    for client_info in client_data:
        client_likes, client_dislikes = client_info
        allIngredients = allIngredients.union(client_likes)
        allIngredients = allIngredients.union(client_dislikes)
    return sorted(list(allIngredients))


In [6]:
def getAllDislikedIngredients():
    "Get all disliked ingredients"
    global client_data
    result = set()
    for client_info in client_data:
        client_likes, client_dislikes = client_info
        result = result.union(client_dislikes)
    return result


def getIngredientsThatWereNeverDisliked():
    "Get all ingredients that were never disliked by any of the clients"
    global client_data
    allIngredients = set(getAllIngredients())
    allDislikes = getAllDislikedIngredients()

    result = set()
    for ingredient in allIngredients:
        if ingredient not in allDislikes:
            result.add(ingredient)

    return result


def getAllIngredientsThatWereDislikedAtleastOnce():
    "Get all ingredients that were disliked by any one of the clients i.e. disliked at least once"
    global client_data
    allIngredients = set(getAllIngredients())
    neverDislikedIngredients = getIngredientsThatWereNeverDisliked()
    return sorted(list(allIngredients.difference(neverDislikedIngredients)))


In [7]:
# This function helps convert the genome list representation into list of ingredients
def genome_to_ingredients(genome: list[int]):
    "Convert genome to what pizza ingredients they represent"
    global allIngredients
    ingredients = set()
    for idx, bit in enumerate(genome):
        if bit == 1:
            ingredients.add(allIngredients[idx])
    return ingredients


# Scoring/Fitness Function
# This function scores the genome i.e. no. of clients satisfied, which we will use for fitness
def get_genome_score(genome):
    "Get score of genome i.e. how many clients were fine with the pizza"
    ingredients = genome_to_ingredients(genome)
    return score_pizza(ingredients)


In [9]:
files = ["./input/" + file for file in os.listdir("./input")]

# Change index here to change the input
input_file = files[3]

client_data = parse_input(input_file)
# allIngredients = list(getAllIngredientsThatWereDislikedAtleastOnce())
allIngredients = list(getAllIngredients())

ingredientCount = len(allIngredients)
clientCount = len(client_data)

bestGenome = None
bestScore = 0

print(f"File selected: {input_file}")
print(f"No. of ingredients: {ingredientCount} | No. of clients: {clientCount}")
print(f"No. of permutations possible: 2^{ingredientCount} or 10^{round(log10(2)*ingredientCount, 2)}")

parameters = {}

parameters["input_filename"] = input_file
parameters["total_ingredient_count"] = ingredientCount
parameters["client_count"] = clientCount


File selected: ./input/d_difficult.in.txt
No. of ingredients: 600 | No. of clients: 9368
No. of permutations possible: 2^600 or 10^180.62
